In [1]:
import numpy as np
import pandas as pd
import csv
import math
import pickle
import pattern_count
import time

"""
cleanAdult = pd.read_csv("CleanAdult2.csv")
miss_class = pd.read_csv("miss_class2.csv")

predictors = ['age', 'workclass', 'education', 'educational-num',
              'marital-status', 'occupation', 'relationship', 'race', 'gender',
              'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']  # len =13

"""




'\ncleanAdult = pd.read_csv("CleanAdult2.csv")\nmiss_class = pd.read_csv("miss_class2.csv")\n\npredictors = [\'age\', \'workclass\', \'education\', \'educational-num\',\n              \'marital-status\', \'occupation\', \'relationship\', \'race\', \'gender\',\n              \'capital-gain\', \'capital-loss\', \'hours-per-week\', \'native-country\']  # len =13\n\n'

In [2]:
def P1DominatedByP2(P1, P2):
    length = len(P1)
    for i in range(length):
        if P1[i] == -1:
            if P2[i] != -1:
                return False
        if P1[i] != -1:
            if P2[i] != P1[i] and P2[i] != -1:
                return False
    return True

# whether a pattern P is dominated by MUP M
def PDominatedByM(P, M):
    for m in M:
        if P1DominatedByP2(P, m):
            return True
    return False

# whether a pattern P dominates MUP M
def PDominatesM(P, M):
    for m in M:
        if P1DominatedByP2(m, P):
            return True
    return False

"""
# coverage of P among dataset D
def cov(P, D):
    cnt = 0
    for d in D:
        if P1DominatedByP2(d, P):
            cnt += 1
    return cnt

"""
def GenerateParents(P):
    parents = []
    length = len(P)
    for i in range(length):
        if P[i] != -1:
            q = P.copy()
            q[i] = -1
            parents.append(q)
    return parents

def GenerateByRule1(P, mcdes, attributes):
    children = []
    length = len(P)
    i = 0
    for i in range(length-1, -1, -1):
        if P[i] != -1:
            break
    if P[i] == -1:
        i -= 1
    for j in range(i+1, length, 1):
        for a in range(int(mcdes[attributes[j]]['min']), int(mcdes[attributes[j]]['max'])+1):
            s = P.copy()
            s[j] = a
            children.append(s)
    return children


def num2string(pattern):
    st = ''
    for i in pattern:
        if i != -1:
            st += str(i)
        st += '|'
    st = st[:-1]
    return st



def Deepdiver(file, tao):
    mc, mcdes, attributes = Prepatation(file)
    column_list = np.array(mc.columns).tolist()
    pc = pattern_count.PatternCounter(file, column_list, encoded=False)
    pc.parse_data()
    
    root = [-1] * (len(attributes))
    S = [root]  # initial stack
    M = []  # maximal uncovered patterns
    while len(S) > 0:
        P = S.pop()
        #print(P, len(M))
        if PDominatedByM(P, M):
            continue
        elif PDominatesM(P, M):
            uncoveredFlag = True
        else:
            st = num2string(P)
            cnt = pc.pattern_count(st)
            uncoveredFlag = cnt < tao and cnt > 0 # add > 0 requirement here
        if uncoveredFlag:
            S1 = [P] # stack
            while len(S1) > 0:
                P1 = S1.pop()
                ParentNodes = GenerateParents(P1)
                for parent in ParentNodes:
                    st = num2string(parent)
                    cntparent = pc.pattern_count(st)
                    if cntparent < tao:
                        S1.append(parent)
                        break
                M.append(P)
            # end while
        else:
            children = GenerateByRule1(P, mcdes, attributes)
            S = S + children
    return M


def Prepatation(filename):
    mc = pd.read_csv(filename)
    mcdes = mc.describe()
    attributes = mcdes.columns.values
    return mc, mcdes, attributes


In [3]:

mc, mcdes, attributes = Prepatation('miss_class3.csv')
print(len(attributes))
mcarray = np.array(mc)
mclist = mcarray.tolist()


FileNotFoundError: [Errno 2] No such file or directory: 'miss_class3.csv'

In [20]:
print(mcdes)

               age    workclass    education  educational-num  marital-status  \
count  4340.000000  4340.000000  4340.000000      4340.000000     4340.000000   
mean      0.657604     2.242166    10.715207        10.805760        2.131106   
std       0.627994     1.068142     3.174135         2.408848        1.069858   
min       0.000000     0.000000     0.000000         1.000000        0.000000   
25%       0.000000     2.000000     9.000000         9.000000        2.000000   
50%       1.000000     2.000000    11.000000        10.000000        2.000000   
75%       1.000000     2.000000    12.000000        13.000000        2.000000   
max       3.000000     6.000000    15.000000        16.000000        6.000000   

        occupation  relationship         race       gender  capital-gain  \
count  4340.000000   4340.000000  4340.000000  4340.000000   4340.000000   
mean      6.300000      0.858065     3.743088     0.808065      0.146544   
std       4.157129      1.605517     0.776

In [21]:
time1 = time.time()
M = Deepdiver('miss_class3.csv', 50*0.6)
time2 = time.time()
print("--- %s seconds ---" % (time2 - time1))
print(M)

KeyboardInterrupt: 

In [5]:
print(len(M)) 

NameError: name 'M' is not defined

In [24]:
with open('M30.pkl', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(M, filehandle)

NameError: name 'M' is not defined

In [25]:
with open('M30.pkl', 'rb') as filehandle:
    # read the data as binary data stream
    placesList = pickle.load(filehandle)

EOFError: Ran out of input